In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from util import DenseTransformer
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, cross_validate
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from tqdm import tqdm


In [3]:
df = pd.read_csv('./data/dataset_with_content.csv', index_col=0, parse_dates=True)

In [4]:
df.head()

,l,close,d,date,pct_change,pt,t,content
next_day,,,,,,,,
2006-01-11,/newsevents/pressreleases/bcreg20060110a.htm,1294.1801,1/10/2006 10:00:00 AM,2006-01-10 10:00:00,0.006314,Banking and Consumer Regulatory Policy,Federal Bank and Thrift Agencies Propose Guida...,\nThe federal bank and thrift regulatory agenc...
2006-02-03,/newsevents/pressreleases/bcreg20060203a.htm,1264.0300,2/3/2006,2006-02-03 00:00:00,-0.000783,Banking and Consumer Regulatory Policy,Federal Financial Regulatory Agencies Issue In...,\nThe federal financial regulatory agencies to...
2006-02-06,/newsevents/pressreleases/bcreg20060206a.htm,1265.0200,2/6/2006,2006-02-06 00:00:00,0.008161,Banking and Consumer Regulatory Policy,Approval of final rule on risk-based capital t...,\nThe Federal Reserve Board on Monday announce...
2006-02-17,/newsevents/pressreleases/bcreg20060217a.htm,1287.2400,2/17/2006,2006-02-17 00:00:00,0.003281,Banking and Consumer Regulatory Policy,Federal Financial Regulatory Agencies Extend C...,\nThe federal financial regulatory agencies to...
2006-02-24,/newsevents/pressreleases/bcreg20060224a.htm,1289.4301,2/24/2006,2006-02-24 00:00:00,-0.003624,Banking and Consumer Regulatory Policy,Federal Regulators Release Summary Findings of...,"\nWASHINGTON, D.C. - The federal bank and thri..."


In [5]:
df.content = df.content.fillna('')

In [8]:
X = df.t +'\n'+ df.content
y = df['pct_change'] > 0

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [11]:
def evaluate_model(model, X=X_train, y=y_train):
    text_svm = make_pipeline(
        CountVectorizer(),
        TfidfTransformer(),
        DenseTransformer(),
        model
    )
#     kf = KFold(n_splits=5)
#     for train_idx, test_idx in kf.split(X):
    
    scores = cross_val_score(text_svm, X, y, cv=5)
    return scores

In [32]:
evaluate_model(GaussianNB())

[0.48221344 0.52173913 0.50988142 0.53359684 0.47920792]


array([0.48221344, 0.52173913, 0.50988142, 0.53359684, 0.47920792])

In [33]:
evaluate_model(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None))

[0.55335968 0.54347826 0.52371542 0.52766798 0.51089109]


array([0.55335968, 0.54347826, 0.52371542, 0.52766798, 0.51089109])

In [36]:
evaluate_model(SVC(random_state=42))

/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packa

[0.54150198 0.54150198 0.54150198 0.54150198 0.54257426]


array([0.54150198, 0.54150198, 0.54150198, 0.54150198, 0.54257426])

In [12]:
evaluate_model(XGBClassifier())

array([0.51976285, 0.54150198, 0.5513834 , 0.48418972, 0.54455446])

In [31]:
def evaluate_model(model, X=X_train, y=y_train):
    text_model = make_pipeline(
        CountVectorizer(),
        TfidfTransformer(),
        DenseTransformer(),
        model
    )
    result = cross_validate(text_model, X,y, cv=5, n_jobs=-1, return_estimator=True)
    return result

In [39]:
result = evaluate_model(XGBClassifier(n_estimators=1000, max_depth=2, n_jobs=-1, random_state=42)); result

/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([354.70518684, 352.3257575 , 355.98370767, 347.36889791,
        351.95765376]),
 'score_time': array([0.24381185, 0.3111732 , 0.2257154 , 0.33550477, 0.31494427]),
 'estimator': (Pipeline(memory=None,
       steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
         seed=None, silent=True, subsample=1))]), Pipeline(memory=None,
       steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...ate=42, reg_alph

In [ ]:
def evaluate_model(model, X=X_train, y=y_train):
    text_model = make_pipeline(
#         CountVectorizer(ngram_range=(1,2)),
        TfidfVectorizer(use_idf=True, ngram_range=(1,2), stop_words='english'),
        DenseTransformer(),
        model
    )
    result = cross_validate(text_model, X,y, cv=5, n_jobs=-1, return_estimator=True)
    return result

tree = XGBClassifier(n_estimators=100, max_depth=3, n_jobs=-1, random_state=42)
result = evaluate_model(); result

In [ ]:
evaluate_model(SVC(random_state=42))